In [1]:
!conda env list

# conda environments:
#
base                  *  /users/avanti/anaconda3
fixed_env                /users/avanti/anaconda3/envs/fixed_env
py2diabeticretinopathy     /users/avanti/anaconda3/envs/py2diabeticretinopathy
py35                     /users/avanti/anaconda3/envs/py35
py376                    /users/avanti/anaconda3/envs/py376



In [2]:
import keras
from keras.models import load_model

model = load_model("/mnt/lab_data2/avanti/ExPecto/resources/deepseabeluga_keras.h5")
model.summary()

Using TensorFlow backend.
/users/avanti/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/users/avanti/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/users/avanti/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/users/avanti/anaconda3/lib/python3.7/site-packages/t

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1993, 320)         10560     
_________________________________________________________________
activation_1 (Activation)    (None, 1993, 320)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1986, 320)         819520    
_________________________________________________________________
activation_2 (Activation)    (None, 1986, 320)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1986, 320)         0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 496, 320)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 489, 480)          1229280   
__________

/users/avanti/anaconda3/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
from keras.models import Sequential
from keras.layers.core import Dense,Flatten
from keras.layers.convolutional import Conv1D

new_model = Sequential()
new_model.add(Conv1D(kernel_size=8, filters=320, input_shape=(2000,4)))
new_model.layers[0].set_weights(model.layers[0].get_weights())
#We copy over remaining layers up until the Permute layer, which we are
# going to get rid of the need for (I had the Permute layer to cope
# with the fact that the original pytorch model was in a 'channels first'
# format, but Keras is 'channels last', and the weights of the pytorch Dense
# layer were such that it was expecting the output of the conv layer to
# be in a 'channels first' format - however, we will deal with this by
# just rearranging the weights of the Dense layer, since we have to modify
# those weights anyway).
for layer in model.layers[1:16]:
  layer_config = layer.get_config()
  del layer_config['name'] #let the name be autogenerated to avoid collisions
  new_layer = type(layer)(**layer_config)
  if (len(layer.get_weights()) > 0):
      new_layer.build(layer.input_shape)
  new_layer.set_weights(layer.get_weights())
  new_model.add(new_layer)
new_model.add(Flatten())
new_model.add(Dense(2003,name="altereddense"))
for layer in model.layers[20:]:
  layer_config = layer.get_config()
  del layer_config['name'] #let the name be autogenerated to avoid collisions
  new_layer = type(layer)(**layer_config)
  if (len(layer.get_weights()) > 0):
      new_layer.build(layer.input_shape)
  new_layer.set_weights(layer.get_weights())
  new_model.add(new_layer)

first_dense_layer_weights, first_dense_layer_biases = model.layers[19].get_weights()
#These next lines alter the weights of the dense layer so that the whole model
# will work on an input of length 400bp
#While porting the weights over, we also get rid of the need for the Permute
# layer by applying the 'transpose' on the weights. The truncation of 50:56 is
# what gets the model to work on an input of length 400bp
new_first_dense_layer_weights = first_dense_layer_weights.reshape((640,106,2003))[:,:,:].transpose((1,0,2)).reshape((640*106,2003))
new_model.layers[17].set_weights([new_first_dense_layer_weights, first_dense_layer_biases])

In [4]:
import numpy as np
random_inputs = np.random.RandomState(1).rand(20,2000,4)

In [5]:
new_preds = new_model.predict(random_inputs[:,:,:])

In [6]:
old_preds = model.predict(random_inputs[:,:,:])

In [7]:
old_preds

array([[0.01404548, 0.00812003, 0.03995261, ..., 0.0014019 , 0.00124151,
        0.00027382],
       [0.01368925, 0.00793654, 0.03913379, ..., 0.00138   , 0.00120765,
        0.00026703],
       [0.01411042, 0.00824639, 0.04028746, ..., 0.00141218, 0.00125223,
        0.00027704],
       ...,
       [0.01376671, 0.00773171, 0.03907946, ..., 0.00139257, 0.00119784,
        0.00026679],
       [0.01378527, 0.0078198 , 0.03910923, ..., 0.00137195, 0.00119868,
        0.00026551],
       [0.01403785, 0.00804397, 0.03988525, ..., 0.00137761, 0.00121778,
        0.00026941]], dtype=float32)

In [8]:
new_preds

array([[0.01404554, 0.00812   , 0.03995261, ..., 0.00140196, 0.00124151,
        0.00027382],
       [0.01368925, 0.00793654, 0.03913385, ..., 0.00138   , 0.00120765,
        0.00026694],
       [0.01411036, 0.00824639, 0.04028741, ..., 0.00141218, 0.00125223,
        0.00027713],
       ...,
       [0.01376671, 0.00773171, 0.03907952, ..., 0.0013926 , 0.00119784,
        0.0002667 ],
       [0.0137853 , 0.0078198 , 0.03910923, ..., 0.00137192, 0.00119859,
        0.00026551],
       [0.01403782, 0.00804397, 0.03988525, ..., 0.00137761, 0.00121778,
        0.00026935]], dtype=float32)

In [9]:
np.max(np.abs(old_preds-new_preds))

1.7881393e-07

In [10]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1993, 320)         10560     
_________________________________________________________________
activation_1 (Activation)    (None, 1993, 320)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1986, 320)         819520    
_________________________________________________________________
activation_2 (Activation)    (None, 1986, 320)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1986, 320)         0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 496, 320)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 489, 480)          1229280   
__________

In [11]:
new_model.save("/mnt/lab_data2/avanti/ExPecto/resources/deepseabeluga_keras_nopermutelayer.h5")